# 07 compare the different variations run in 05a
- alternative reference regions and/or changes in subject inclusion

versions needed to compare:

Data versions:
original reference region + original subject set
Paracentral-gm reference region + original subject set
Paracentral-gm reference region + new subject set
RAS-ref1 reference region + original subject set
RAS-ref1 reference region + new subject set

In [2]:
# install required packages - commented out so it doesn't install every time
#%conda install -n Lauren openpyxl numpy pandas statsmodels plotnine matplotlib scikit-learn scipy mizani nbconvert pandoc pyreadstat kmodes seaborn

# import required packages
import numpy as np;
import pandas as pd;
from sklearn import preprocessing 
import statsmodels as sm;
import statsmodels.formula.api as smf;
import plotnine as p9;
import pickle



# read in and clean data

In [3]:
data_folder_loc = "//admsyn/Primary/ADM/CustomerStudies/Rockefeller/Riluzole_Biomarkers/"
code_folder_loc = "//admsyn/homes/@DH-ADMDX/0/lauren.koenig-1606/code/Riluzole FDG/"

In [4]:
x_var_list = ['Ab42_40', 'GFAP', 'NFL', 'pTau181', 'pTau217', 'pTau231', 'pTau217_Ab42']
y_var_list = ['RASref1_gm', 'Paracentral_gm', 'Ab42_log10', 'pTau181_Ab42_log10', 'Ab40_log10', 'pTau217_Ab42_log10']

rename_columns = ['model_p_val_baseline', 'model_B_val_baseline',
'model_n_baseline', 
       'model_p_val_Intercept', 'model_B_val_Intercept',
       'model_p_val_x_var', 'model_B_val_x_var',
        'model_p_val_age','model_B_val_age',
       'model_p_val_Education_years', 'model_B_val_Education_years', 
       'model_p_val_sex[T.M]', 'model_B_val_sex[T.M]',
        'model_p_val_apoe4_carrier[T.Yes]', 'model_B_val_apoe4_carrier[T.Yes]',
         'model_p_val_race[T.White/NH]', 'model_B_val_race[T.White/NH]']

covariate_pval_columns = [ 'model_p_val_age',
       'model_p_val_Education_years', 
       'model_p_val_sex[T.M]', 
        'model_p_val_apoe4_carrier[T.Yes]',
         'model_p_val_race[T.White/NH]']


# define biomarker groups
FDG_columns = ['Avg_MedOrbFrontal', 'Graycer_gm', 'Avg_PCC', 'Avg_Hip', 'MTL_gm', 'CO', 'Temp', 'SensMot_gm', 'L_Hip',
 'Vermis_gm', 'FRONTAL_gm', 'AC_gm', 'Precun_gm', 'Par_gm', 'RASref1_gm', 'Temp_gm', 'Paracentral_gm', 'R_Hip', 'PostCing_gm']

plasma_columns = [ 'Ab40', 'Ab42', 'Ab42_40', 'GFAP','NFL',  'pTau181', 'pTau217',  'pTau231', 'pTau181_Ab42', 'GFAP_aa', 'NFL_aa', 'pTau217_Ab42']
log10_plasma_columns = [s + '_log10' for s in plasma_columns]

cog_columns = ['MMSE', 'adascogtotal', 'bvrt', 'dstotal','tma','tmb',  'cowattotal', 'cdrtotal', 'cdrsum', 'adltotal', 'npitotal', 'gds']
vol_columns = ['VOL_Precun_Lz', 'VOL_Precun_Rz', 'VOL_InfPar_Lz', 'VOL_InfPar_Rz', 'VOL_Hip_Lz', 'VOL_Hip_Rz', 'VOL_TotalGrayz', 'VOL_Inf_Mid_Fus_Temp_Lz', 
'VOL_Inf_Mid_Fus_Temp_Rz','VOL_MidFront_Lz', 'VOL_MidFront_Rz']
      

In [5]:
# read in data
results_new_para, temp, data4 = pd.read_pickle(code_folder_loc + '/output/05a_data_new_para_rr.pkl')
results_new_para = results_new_para[results_new_para['x_var'].isin(x_var_list) & ~results_new_para['y_var'].isin(y_var_list)]
results_new_para = results_new_para.rename(columns = dict(zip(rename_columns, 
['new_para_' + ele for ele in rename_columns])))

# read in data
results_pons, temp, data2 = pd.read_pickle(code_folder_loc + '/output/05a_data_pons_rr.pkl')
results_pons = results_pons[results_pons['x_var'].isin(x_var_list) & ~results_pons['y_var'].isin(y_var_list)]
results_pons = results_pons.rename(columns = dict(zip(rename_columns, 
['pons_' + ele for ele in rename_columns])))

'''
# read in data
results_composite, temp, data1 = pd.read_pickle(code_folder_loc + '/output/05a_data_composite_rr.pkl')
results_composite = results_composite[results_composite['x_var'].isin(x_var_list) & ~results_composite['y_var'].isin(y_var_list)]
results_composite = results_composite.rename(columns = dict(zip(rename_columns, 
['composite_' + ele for ele in rename_columns])))

# read in data
results_para, temp, data3 = pd.read_pickle(code_folder_loc + '/output/05a_data_para_rr.pkl')
results_para = results_para[results_para['x_var'].isin(x_var_list) & ~results_para['y_var'].isin(y_var_list)]
results_para = results_para.rename(columns = dict(zip(rename_columns, 
['para_' + ele for ele in rename_columns])))

# read in data
results_ras, temp, data5 = pd.read_pickle(code_folder_loc + '/output/05a_data_ras_rr.pkl')
results_ras = results_ras[results_ras['x_var'].isin(x_var_list) & ~results_ras['y_var'].isin(y_var_list)]
results_ras = results_ras.rename(columns = dict(zip(rename_columns, 
['ras_' + ele for ele in rename_columns])))

# read in data
results_subset_new_para, temp, data5_subset = pd.read_pickle(code_folder_loc + '/output/05a_data_new_para_rr_subset.pkl')
results_subset_new_para = results_subset_new_para[results_subset_new_para['x_var'].isin(x_var_list) & ~results_subset_new_para['y_var'].isin(y_var_list)]
results_subset_new_para = results_subset_new_para.rename(columns = dict(zip(rename_columns, 
['subset_new_para_' + ele for ele in rename_columns])))

# read in data
results_orig_new_para, temp, data6 = pd.read_pickle(code_folder_loc + '/output/with_RIL8_included/05a_data_new_para_rr.pkl')
results_orig_new_para = results_orig_new_para[results_orig_new_para['x_var'].isin(x_var_list) & ~results_orig_new_para['y_var'].isin(y_var_list)]
results_orig_new_para = results_orig_new_para.rename(columns = dict(zip(rename_columns, 
['orig_new_para_' + ele for ele in rename_columns])))
'''

"\n# read in data\nresults_composite, temp, data1 = pd.read_pickle(code_folder_loc + '/output/05a_data_composite_rr.pkl')\nresults_composite = results_composite[results_composite['x_var'].isin(x_var_list) & ~results_composite['y_var'].isin(y_var_list)]\nresults_composite = results_composite.rename(columns = dict(zip(rename_columns, \n['composite_' + ele for ele in rename_columns])))\n\n# read in data\nresults_para, temp, data3 = pd.read_pickle(code_folder_loc + '/output/05a_data_para_rr.pkl')\nresults_para = results_para[results_para['x_var'].isin(x_var_list) & ~results_para['y_var'].isin(y_var_list)]\nresults_para = results_para.rename(columns = dict(zip(rename_columns, \n['para_' + ele for ele in rename_columns])))\n\n# read in data\nresults_ras, temp, data5 = pd.read_pickle(code_folder_loc + '/output/05a_data_ras_rr.pkl')\nresults_ras = results_ras[results_ras['x_var'].isin(x_var_list) & ~results_ras['y_var'].isin(y_var_list)]\nresults_ras = results_ras.rename(columns = dict(zip(ren

In [ ]:
results_new_para.groupby('x_var')

,y_var,x_var,new_para_model_p_val_baseline,new_para_model_B_val_baseline,new_para_model_n_baseline,new_para_model_p_val_Intercept,new_para_model_p_val_Education_years,new_para_model_p_val_x_var,new_para_model_B_val_Intercept,new_para_model_B_val_Education_years,new_para_model_B_val_x_var,new_para_model_p_val_age,new_para_model_B_val_age,new_para_model_p_val_race[T.White/NH],new_para_model_B_val_race[T.White/NH],new_para_model_p_val_sex[T.M],new_para_model_B_val_sex[T.M],new_para_model_p_val_apoe4_carrier[T.Yes],new_para_model_B_val_apoe4_carrier[T.Yes],type
0,MTL_gm,Ab42_40,Intercept 0.917548 Education_years ...,Intercept -0.014719 Education_years ...,35,0.917548,0.002342,0.010588,-1.471869e-02,0.467310,0.389317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FDG PET
0,SensMot_gm,Ab42_40,Intercept 1.000000 Ab42_40_log10 0.8...,Intercept -1.285430e-15 Ab42_40_log10 ...,36,1.000000,NaN,0.816615,-1.285430e-15,NaN,0.040048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FDG PET
0,FRONTAL_gm,Ab42_40,Intercept 1.000000 Ab42_40_log10 0.7...,Intercept -3.358425e-15 Ab42_40_log10 ...,36,1.000000,NaN,0.742813,-3.358425e-15,NaN,-0.056644,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FDG PET
0,AC_gm,Ab42_40,Intercept 1.000000 Ab42_40_log10 0.8...,Intercept 3.191891e-16 Ab42_40_log10 ...,36,1.000000,NaN,0.823329,3.191891e-16,NaN,0.038558,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FDG PET
0,Precun_gm,Ab42_40,Intercept 0.911983 age ...,Intercept -0.016633 age ...,35,0.911983,0.024805,0.021403,-1.663288e-02,0.364864,0.373903,0.007766,0.447645,NaN,NaN,NaN,NaN,NaN,NaN,FDG PET
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,VOL_Precun_InfPar_Supramarg_Rz,pTau217_Ab42,Intercept 1.000000 pTau217_Ab42_lo...,Intercept -1.249001e-16 pTau217_Ab4...,27,1.000000,NaN,0.650067,-1.249001e-16,NaN,-0.091453,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Volumetric
0,VOL_LatOccLingCun_Lz,pTau217_Ab42,Intercept 1.000000 pTau217_Ab42_lo...,Intercept -3.469447e-17 pTau217_Ab4...,27,1.000000,NaN,0.746738,-3.469447e-17,NaN,0.065168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Volumetric
0,VOL_LatOccLingCun_Rz,pTau217_Ab42,Intercept 1.000000 pTau217_Ab42_lo...,Intercept 5.551115e-17 pTau217_Ab4...,27,1.000000,NaN,0.635484,5.551115e-17,NaN,0.095537,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Volumetric
0,VOL_InfParSupra_Lz,pTau217_Ab42,Intercept 1.000000 age ...,Intercept 4.163336e-17 age ...,27,1.000000,NaN,0.735875,4.163336e-17,NaN,-0.062011,0.010779,0.502351,NaN,NaN,NaN,NaN,NaN,NaN,Volumetric


In [21]:
results_new_para.groupby('x_var')[[ 'new_para_model_p_val_x_var','new_para_model_p_val_Education_years',
       'new_para_model_p_val_age', 
       'new_para_model_p_val_race[T.White/NH]',
       'new_para_model_p_val_sex[T.M]', 
       'new_para_model_p_val_apoe4_carrier[T.Yes]']].apply(lambda x: x.notna().sum())

,new_para_model_p_val_x_var,new_para_model_p_val_Education_years,new_para_model_p_val_age,new_para_model_p_val_race[T.White/NH],new_para_model_p_val_sex[T.M],new_para_model_p_val_apoe4_carrier[T.Yes]
x_var,,,,,,
Ab42_40,107,6,15,0,0,0
GFAP,107,2,18,0,0,1
NFL,107,2,17,0,1,1
pTau181,107,8,24,0,2,1
pTau217,107,3,11,0,2,1
pTau217_Ab42,107,6,13,0,1,0
pTau231,107,10,26,0,4,0


# Pons vs New Para

In [5]:
comp_new_para = pd.merge(results_pons, results_new_para, how = 'inner')
comp_new_para

,y_var,x_var,pons_model_p_val_baseline,pons_model_B_val_baseline,pons_model_n_baseline,pons_model_p_val_Intercept,pons_model_p_val_x_var,pons_model_B_val_Intercept,pons_model_B_val_x_var,pons_model_p_val_sex[T.M],...,new_para_model_B_val_Education_years,new_para_model_B_val_x_var,new_para_model_p_val_age,new_para_model_B_val_age,new_para_model_p_val_race[T.White/NH],new_para_model_B_val_race[T.White/NH],new_para_model_p_val_sex[T.M],new_para_model_B_val_sex[T.M],new_para_model_p_val_apoe4_carrier[T.Yes],new_para_model_B_val_apoe4_carrier[T.Yes]
0,MTL_gm,Ab42_40,Intercept 1.000000 Ab42_40_log10 0.5...,Intercept 1.291502e-15 Ab42_40_log10 ...,31,1.000000,0.538983,1.291502e-15,0.114689,NaN,...,0.467310,0.389317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AC_gm,Ab42_40,Intercept 0.024900 sex[T.M] 0.0...,Intercept 0.464529 sex[T.M] -1.2...,31,0.024900,0.655576,4.645291e-01,-0.072157,0.001059,...,NaN,0.038558,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Precun_gm,Ab42_40,Intercept 0.100797 sex[T.M] 0.0...,Intercept 0.377718 sex[T.M] -0.9...,31,0.100797,0.269296,3.777183e-01,0.204866,0.014219,...,0.364864,0.373903,0.007766,0.447645,NaN,NaN,NaN,NaN,NaN,NaN
3,Par_gm,Ab42_40,Intercept 1.000000 Ab42_40_log10 0.9...,Intercept -1.200429e-15 Ab42_40_log10 ...,31,1.000000,0.901895,-1.200429e-15,-0.023085,NaN,...,NaN,0.192024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Temp_gm,Ab42_40,Intercept 0.106899 sex[T.M] 0.0...,Intercept 0.372288 sex[T.M] -0.9...,31,0.106899,0.390890,3.722882e-01,0.159058,0.015902,...,NaN,0.293636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730,VOL_Precun_InfPar_Supramarg_Rz,pTau217_Ab42,Intercept 1.000000 pTau217_Ab42_lo...,Intercept -1.387779e-16 pTau217_Ab4...,27,1.000000,0.650067,-1.387779e-16,-0.091453,NaN,...,NaN,-0.091453,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
731,VOL_LatOccLingCun_Lz,pTau217_Ab42,Intercept 1.000000 pTau217_Ab42_lo...,Intercept 5.551115e-17 pTau217_Ab4...,27,1.000000,0.746738,5.551115e-17,0.065168,NaN,...,NaN,0.065168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
732,VOL_LatOccLingCun_Rz,pTau217_Ab42,Intercept 1.000000 pTau217_Ab42_lo...,Intercept -4.336809e-17 pTau217_Ab4...,27,1.000000,0.635484,-4.336809e-17,0.095537,NaN,...,NaN,0.095537,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
733,VOL_InfParSupra_Lz,pTau217_Ab42,Intercept 1.000000 age ...,Intercept 1.387779e-17 age ...,27,1.000000,0.735875,1.387779e-17,-0.062011,NaN,...,NaN,-0.062011,0.010779,0.502351,NaN,NaN,NaN,NaN,NaN,NaN


## x_var significance

In [6]:
comp_new_para.loc[(comp_new_para['pons_model_p_val_x_var'] < 0.05) & (comp_new_para['new_para_model_p_val_x_var'] >= 0.05), ['x_var', 'y_var', 
'pons_model_p_val_x_var', 'new_para_model_p_val_x_var',
'pons_model_B_val_x_var', 'new_para_model_B_val_x_var', 
'pons_model_n_baseline','new_para_model_n_baseline']]

,x_var,y_var,pons_model_p_val_x_var,new_para_model_p_val_x_var,pons_model_B_val_x_var,new_para_model_B_val_x_var,pons_model_n_baseline,new_para_model_n_baseline


In [7]:
comp_new_para.loc[(comp_new_para['pons_model_p_val_x_var'] >= 0.05) & (comp_new_para['new_para_model_p_val_x_var'] < 0.05), 
['x_var', 'y_var',
'pons_model_p_val_x_var', 'new_para_model_p_val_x_var',
'pons_model_B_val_x_var', 'new_para_model_B_val_x_var', 
'pons_model_n_baseline','new_para_model_n_baseline']]

,x_var,y_var,pons_model_p_val_x_var,new_para_model_p_val_x_var,pons_model_B_val_x_var,new_para_model_B_val_x_var,pons_model_n_baseline,new_para_model_n_baseline
0,Ab42_40,MTL_gm,0.538983,0.010588,0.114689,0.389317,31,35
2,Ab42_40,Precun_gm,0.269296,0.021403,0.204866,0.373903,31,35
18,pTau181,MTL_gm,0.103894,0.000048,0.271625,0.547130,37,41
23,pTau181,PostCing_gm,0.281572,0.015868,0.159634,0.373023,37,41
26,pTau217,Precun_gm,0.240472,0.044767,-0.206857,-0.327454,34,38
30,pTau231,MTL_gm,0.164072,0.032529,0.247989,0.347523,33,38
39,pTau217_Ab42,Par_gm,0.123870,0.011858,-0.292311,-0.432901,29,33
40,pTau217_Ab42,Temp_gm,0.146871,0.016024,-0.276260,-0.416077,29,33


In [8]:
(comp_new_para['pons_model_p_val_x_var']  - comp_new_para['new_para_model_p_val_x_var']).describe().round(3)

count    735.000
mean       0.010
std        0.088
min       -0.598
25%       -0.000
50%        0.000
75%        0.000
max        0.924
dtype: float64

In [9]:
comp_new_para.loc[(abs(comp_new_para['pons_model_p_val_x_var']  - comp_new_para['new_para_model_p_val_x_var'])> 0.1), ['x_var', 'y_var', 'pons_model_p_val_x_var',
       'pons_model_B_val_x_var', 'pons_model_n_baseline', 'new_para_model_p_val_x_var', 'new_para_model_B_val_x_var',
       'new_para_model_n_baseline']].sort_values('new_para_model_p_val_x_var')

,x_var,y_var,pons_model_p_val_x_var,pons_model_B_val_x_var,pons_model_n_baseline,new_para_model_p_val_x_var,new_para_model_B_val_x_var,new_para_model_n_baseline
18,pTau181,MTL_gm,0.103894,0.271625,37,0.000048,0.547130,41
0,Ab42_40,MTL_gm,0.538983,0.114689,31,0.010588,0.389317,35
39,pTau217_Ab42,Par_gm,0.123870,-0.292311,29,0.011858,-0.432901,33
23,pTau181,PostCing_gm,0.281572,0.159634,37,0.015868,0.373023,41
40,pTau217_Ab42,Temp_gm,0.146871,-0.276260,29,0.016024,-0.416077,33
2,Ab42_40,Precun_gm,0.269296,0.204866,31,0.021403,0.373903,35
30,pTau231,MTL_gm,0.164072,0.247989,33,0.032529,0.347523,38
26,pTau217,Precun_gm,0.240472,-0.206857,34,0.044767,-0.327454,38
10,GFAP,Temp_gm,0.632654,-0.079818,37,0.061341,-0.291202,42
38,pTau217_Ab42,Precun_gm,0.400128,-0.162343,29,0.062075,-0.328377,33


## covariate significance

In [10]:
# those where covariate significance differs between a and b
a_sig = comp_new_para[['pons_' + ele for ele in covariate_pval_columns]].notna().any(axis =1)
b_sig = comp_new_para[['new_para_' + ele for ele in covariate_pval_columns]].notna().any(axis =1)

temp = comp_new_para[a_sig != b_sig].drop(columns = comp_new_para.columns[comp_new_para.columns.str.contains('B_val|Intercept|baseline|type|p_val_x_var')]).dropna(axis = 1, how = 'all')
temp = temp[temp.columns.sort_values()]
temp = temp.loc[temp.isna().sort_values(temp.columns.to_list()).index, :]
temp

,new_para_model_p_val_Education_years,new_para_model_p_val_age,pons_model_p_val_sex[T.M],x_var,y_var
20,0.006586,0.016969,NaN,pTau181,Precun_gm
0,0.002342,NaN,NaN,Ab42_40,MTL_gm
6,0.005543,NaN,NaN,GFAP,MTL_gm
12,0.001880,NaN,NaN,NFL,MTL_gm
18,0.000017,NaN,NaN,pTau181,MTL_gm
24,0.013116,NaN,NaN,pTau217,MTL_gm
36,0.006126,NaN,NaN,pTau217_Ab42,MTL_gm
9,NaN,0.028485,NaN,GFAP,Par_gm
15,NaN,0.036415,NaN,NFL,Par_gm
21,NaN,0.041938,NaN,pTau181,Par_gm


# New Para GFAP and NfL age adjustment

In [11]:
temp_a, temp, data4 = pd.read_pickle(code_folder_loc + '/output/05a_data_new_para_rr.pkl')
temp_b = temp_a[temp_a['x_var'].isin(['GFAP_aa', 'NFL_aa']) & ~temp_a['y_var'].isin(['GFAP', 'NFL'])]
temp_a = temp_a[temp_a['x_var'].isin(['GFAP', 'NFL'])& ~temp_a['y_var'].isin(['GFAP_aa', 'NFL_aa'])]
temp_a = temp_a.rename(columns = dict(zip(rename_columns, ['new_para_' + ele for ele in rename_columns])))

temp_b['x_var'] = temp_b['x_var'].str.replace('_aa', '')
temp_b = temp_b.rename(columns = dict(zip(rename_columns, ['aa_' + ele for ele in rename_columns])))


In [12]:
comp_new_para = pd.merge(temp_a, temp_b, how = 'inner').sort_values(['x_var', 'type', 'y_var'])
comp_new_para

,y_var,x_var,new_para_model_p_val_baseline,new_para_model_B_val_baseline,new_para_model_n_baseline,new_para_model_p_val_Intercept,new_para_model_p_val_Education_years,new_para_model_p_val_x_var,new_para_model_B_val_Intercept,new_para_model_B_val_Education_years,...,aa_model_B_val_Education_years,aa_model_B_val_x_var,aa_model_p_val_age,aa_model_B_val_age,aa_model_p_val_race[T.White/NH],aa_model_B_val_race[T.White/NH],aa_model_p_val_sex[T.M],aa_model_B_val_sex[T.M],aa_model_p_val_apoe4_carrier[T.Yes],aa_model_B_val_apoe4_carrier[T.Yes]
34,MMSE,GFAP,Intercept 1.000000 GFAP_log10 0.071011 ...,Intercept -5.595970e-16 GFAP_log10 -3.002...,37,1.0,NaN,0.071011,-5.595970e-16,NaN,...,NaN,-0.315879,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,adascogtotal,GFAP,Intercept 1.00000 GFAP_log10 0.10464 dt...,Intercept 1.433858e-16 GFAP_log10 2.710...,37,1.0,NaN,0.104640,1.433858e-16,NaN,...,NaN,0.276279,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,adltotal,GFAP,Intercept 1.000000 GFAP_log10 0.777709 ...,Intercept -2.560232e-16 GFAP_log10 -4.803...,37,1.0,NaN,0.777709,-2.560232e-16,NaN,...,NaN,0.011609,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,bvrt,GFAP,Intercept 1.000000 GFAP_log10 0.426859 ...,Intercept 5.724587e-17 GFAP_log10 -1.366...,36,1.0,NaN,0.426859,5.724587e-17,NaN,...,NaN,-0.141799,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,cdrsum,GFAP,Intercept 1.000000 GFAP_log10 0.050683 ...,Intercept -9.021911e-17 GFAP_log10 3.236...,37,1.0,NaN,0.050683,-9.021911e-17,NaN,...,NaN,0.365860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,VOL_TotalGray_Lz,NFL,Intercept 1.000000 age 0.012554 NF...,Intercept 1.665335e-16 age 4.21065...,36,1.0,NaN,0.647110,1.665335e-16,NaN,...,NaN,0.095707,0.007854,0.467916,NaN,NaN,NaN,NaN,NaN,NaN
189,VOL_TotalGray_Rz,NFL,Intercept 1.000000 NFL_log10 0.745612 dt...,Intercept -1.110223e-16 NFL_log10 5.60057...,36,1.0,NaN,0.745612,-1.110223e-16,NaN,...,NaN,-0.091617,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187,VOL_TotalGrayz,NFL,Intercept 1.000000 age 0.011972 NF...,Intercept -5.551115e-17 age 4.26538...,36,1.0,NaN,0.889038,-5.551115e-17,NaN,...,NaN,0.060240,0.010492,0.451189,NaN,NaN,NaN,NaN,NaN,NaN
135,VOL_Ventricles_Lz,NFL,Intercept 1.000000 NFL_log10 0.218555 dt...,Intercept 1.422473e-16 NFL_log10 2.10181...,36,1.0,NaN,0.218555,1.422473e-16,NaN,...,NaN,0.314744,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
temp_a[temp_a['x_var'].isin(['GFAP']) & temp_a['y_var'].isin(['VOL_ParaHip_Lz', 'VOL_Pericalc_Rz', 'VOL_Hip_Rz'])].dropna(how = 'all', axis = 1)

,y_var,x_var,new_para_model_p_val_baseline,new_para_model_B_val_baseline,new_para_model_n_baseline,new_para_model_p_val_Intercept,new_para_model_p_val_x_var,new_para_model_B_val_Intercept,new_para_model_B_val_x_var,type
0,VOL_ParaHip_Lz,GFAP,Intercept 1.000000 GFAP_log10 0.026352 ...,Intercept 1.457168e-16 GFAP_log10 3.699...,36,1.0,0.026352,1.457168e-16,0.369975,Volumetric
0,VOL_Pericalc_Rz,GFAP,Intercept 1.000000 GFAP_log10 0.025552 ...,Intercept -1.387779e-17 GFAP_log10 3.718...,36,1.0,0.025552,-1.387779e-17,0.371826,Volumetric
0,VOL_Hip_Rz,GFAP,Intercept 1.000000 GFAP_log10 0.045641 ...,Intercept 9.714451e-17 GFAP_log10 3.352...,36,1.0,0.045641,9.714451e-17,0.335238,Volumetric


In [14]:
temp_b[temp_b['x_var'].isin(['GFAP']) & temp_b['y_var'].isin(['VOL_ParaHip_Lz', 'VOL_Pericalc_Rz', 'VOL_Hip_Rz'])].dropna(how = 'all', axis = 1)

,y_var,x_var,aa_model_p_val_baseline,aa_model_B_val_baseline,aa_model_n_baseline,aa_model_p_val_Intercept,aa_model_p_val_x_var,aa_model_B_val_Intercept,aa_model_B_val_x_var,type
0,VOL_ParaHip_Lz,GFAP,Intercept 1.000000 GFAP_aa_log10 0.0...,Intercept 1.457168e-16 GFAP_aa_log10 ...,36,1.0,0.015137,1.457168e-16,0.401803,Volumetric
0,VOL_Pericalc_Rz,GFAP,Intercept 1.000000 GFAP_aa_log10 0.0...,Intercept -1.387779e-17 GFAP_aa_log10 ...,36,1.0,0.036374,-1.387779e-17,0.350022,Volumetric
0,VOL_Hip_Rz,GFAP,Intercept 1.000000 GFAP_aa_log10 0.0...,Intercept 9.714451e-17 GFAP_aa_log10 ...,36,1.0,0.025043,9.714451e-17,0.373027,Volumetric


## x_var significance

In [15]:
comp_new_para.loc[(comp_new_para['aa_model_p_val_x_var'] < 0.05) & (comp_new_para['new_para_model_p_val_x_var'] >= 0.05), ['x_var', 'y_var', 
'aa_model_p_val_x_var', 'new_para_model_p_val_x_var',
'aa_model_B_val_x_var', 'new_para_model_B_val_x_var', 
'aa_model_n_baseline','new_para_model_n_baseline']]

,x_var,y_var,aa_model_p_val_x_var,new_para_model_p_val_x_var,aa_model_B_val_x_var,new_para_model_B_val_x_var,aa_model_n_baseline,new_para_model_n_baseline
42,GFAP,cdrsum,0.025954,0.050683,0.365860,0.323664,37,37
5,GFAP,Par_gm,0.039738,0.073024,-0.322502,-0.271715,41,41
4,GFAP,Precun_gm,0.024812,0.141319,-0.350153,-0.226042,41,41
90,GFAP,VOL_SupFront_Lz,0.003535,0.058331,-0.513773,-0.318501,36,36
78,GFAP,VOL_SupraMarg_Lz,0.037834,0.171286,-0.347503,-0.233085,36,36
52,NFL,cowattotal,0.045767,0.109022,-0.330444,-0.267820,37,37
32,NFL,pTau231,0.017999,0.086283,0.392172,0.278274,36,39


In [16]:
comp_new_para.loc[(comp_new_para['aa_model_p_val_x_var'] >= 0.05) & (comp_new_para['new_para_model_p_val_x_var'] < 0.05), 
['x_var', 'y_var',
'aa_model_p_val_x_var', 'new_para_model_p_val_x_var',
'aa_model_B_val_x_var', 'new_para_model_B_val_x_var', 
'aa_model_n_baseline','new_para_model_n_baseline']]

,x_var,y_var,aa_model_p_val_x_var,new_para_model_p_val_x_var,aa_model_B_val_x_var,new_para_model_B_val_x_var,aa_model_n_baseline,new_para_model_n_baseline
133,GFAP,VOL_InfParSupra_Lz,0.055790,0.038333,-0.287553,-0.300265,36,36
138,NFL,VOL_Putamen_Rz,0.063296,0.038900,-0.339207,-0.359101,31,31


In [17]:
(comp_new_para['aa_model_p_val_x_var']  - comp_new_para['new_para_model_p_val_x_var']).describe().round(3)

count    210.000
mean       0.001
std        0.201
min       -0.716
25%       -0.124
50%       -0.011
75%        0.079
max        0.610
dtype: float64

In [18]:
comp_new_para.loc[(abs(comp_new_para['aa_model_p_val_x_var']  - comp_new_para['new_para_model_p_val_x_var'])> 0.1), ['x_var', 'y_var', 'aa_model_p_val_x_var',
       'aa_model_B_val_x_var', 'aa_model_n_baseline', 'new_para_model_p_val_x_var', 'new_para_model_B_val_x_var',
       'new_para_model_n_baseline']].sort_values('new_para_model_p_val_x_var')

,x_var,y_var,aa_model_p_val_x_var,aa_model_B_val_x_var,aa_model_n_baseline,new_para_model_p_val_x_var,new_para_model_B_val_x_var,new_para_model_n_baseline
28,NFL,Ab42,0.156172,0.244923,35,0.052651,0.316781,38
167,NFL,VOL_SupFront_Lz,0.327457,0.167973,36,0.069684,0.305830,36
149,NFL,VOL_Precun_Lz,0.697386,0.067104,36,0.098531,0.279681,36
200,NFL,VOL_MidSupFrontal_Lz,0.733446,0.058783,36,0.123627,0.261361,36
27,NFL,Ab40,0.344773,0.155420,39,0.139693,0.231775,42
...,...,...,...,...,...,...,...,...
205,NFL,VOL_Precun_InfPar_Rz,0.778385,0.046862,36,0.964764,-0.007087,36
204,NFL,VOL_Precun_InfPar_Lz,0.792964,0.042473,36,0.968283,0.006199,36
155,NFL,VOL_SupraMarg_Lz,0.255369,-0.194616,36,0.971573,0.006157,36
125,GFAP,VOL_Inf_Mid_Fus_Temp_Lz,0.816968,-0.039970,36,0.976664,0.005054,36


## covariate significance

In [19]:
# those where covariate significance differs between a and b
a_sig = comp_new_para[['aa_' + ele for ele in covariate_pval_columns]].notna().any(axis =1)
b_sig = comp_new_para[['new_para_' + ele for ele in covariate_pval_columns]].notna().any(axis =1)

temp = comp_new_para[a_sig != b_sig].drop(columns = comp_new_para.columns[comp_new_para.columns.str.contains('B_val|Intercept|baseline|type|p_val_x_var')]).dropna(axis = 1, how = 'all')
temp = temp[temp.columns.sort_values()]
temp = temp.loc[temp.isna().sort_values(temp.columns.to_list()).index, :]
temp

,aa_model_p_val_age,aa_model_p_val_apoe4_carrier[T.Yes],aa_model_p_val_sex[T.M],new_para_model_p_val_Education_years,new_para_model_p_val_age,x_var,y_var
73,0.007058,NaN,NaN,NaN,NaN,GFAP,VOL_Precun_Rz
150,0.004473,NaN,NaN,NaN,NaN,NFL,VOL_Precun_Rz
39,NaN,0.04734,0.042217,NaN,NaN,GFAP,tmb
90,NaN,NaN,0.049009,NaN,NaN,GFAP,VOL_SupFront_Lz
91,NaN,NaN,0.044263,NaN,NaN,GFAP,VOL_SupFront_Rz
4,NaN,NaN,NaN,0.043796,0.038379,GFAP,Precun_gm
5,NaN,NaN,NaN,NaN,0.028485,GFAP,Par_gm
50,NaN,NaN,NaN,NaN,0.034720,NFL,tma
15,NaN,NaN,NaN,NaN,0.036415,NFL,Par_gm
211,NaN,NaN,NaN,NaN,0.015942,NFL,VOL_InfParSupra_Rz


# New Para ptau 217 vs ptau 217/Ab42

In [20]:
temp_a, temp, data4 = pd.read_pickle(code_folder_loc + '/output/05a_data_new_para_rr.pkl')
temp_b = temp_a[temp_a['x_var'].isin(['pTau217']) | temp_a['y_var'].isin(['pTau217'])]
temp_a = temp_a[temp_a['x_var'].isin(['pTau217_Ab42']) | temp_a['y_var'].isin(['pTau217_Ab42'])]
temp_a = temp_a.rename(columns = dict(zip(rename_columns, ['new_para_' + ele for ele in rename_columns])))

temp_a['x_var'] = temp_a['x_var'].str.replace('pTau217_Ab42', 'pTau217')
temp_a['y_var'] = temp_a['y_var'].str.replace('pTau217_Ab42', 'pTau217')
temp_b = temp_b.rename(columns = dict(zip(rename_columns, ['ratioAb42_' + ele for ele in rename_columns])))


In [21]:
comp_new_para = pd.merge(temp_a, temp_b, how = 'inner').sort_values(['x_var', 'type', 'y_var'])
comp_new_para

,y_var,x_var,new_para_model_p_val_baseline,new_para_model_B_val_baseline,new_para_model_n_baseline,new_para_model_p_val_Intercept,new_para_model_p_val_Education_years,new_para_model_p_val_x_var,new_para_model_B_val_Intercept,new_para_model_B_val_Education_years,...,ratioAb42_model_B_val_Education_years,ratioAb42_model_B_val_x_var,ratioAb42_model_p_val_age,ratioAb42_model_B_val_age,ratioAb42_model_p_val_race[T.White/NH],ratioAb42_model_B_val_race[T.White/NH],ratioAb42_model_p_val_sex[T.M],ratioAb42_model_B_val_sex[T.M],ratioAb42_model_p_val_apoe4_carrier[T.Yes],ratioAb42_model_B_val_apoe4_carrier[T.Yes]
10,pTau217,Ab40,Intercept 1.000000 Ab40_log10 0.000783 ...,Intercept -1.665335e-16 Ab40_log10 -5.484...,34,1.0,NaN,0.000783,-1.665335e-16,NaN,...,NaN,-0.089037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,pTau217,Ab42,Intercept 1.000000 Ab42_log10 0.000142 ...,Intercept -1.665335e-16 Ab42_log10 -6.068...,34,1.0,NaN,0.000142,-1.665335e-16,NaN,...,NaN,-0.138977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,pTau217,Ab42_40,Intercept 1.000000 Ab42_40_log10 0.7...,Intercept -1.665335e-16 Ab42_40_log10 ...,34,1.0,NaN,0.794579,-1.665335e-16,NaN,...,NaN,0.057931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,pTau217,GFAP,Intercept 1.000000 GFAP_log10 0.160224 ...,Intercept -1.665335e-16 GFAP_log10 2.463...,34,1.0,NaN,0.160224,-1.665335e-16,NaN,...,NaN,0.396230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,pTau217,GFAP_aa,Intercept 1.000000 GFAP_aa_log10 0.1...,Intercept -1.249001e-16 GFAP_aa_log10 ...,32,1.0,NaN,0.127015,-1.249001e-16,NaN,...,NaN,0.416667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,VOL_TotalGray_Rz,pTau217,Intercept 1.000000 pTau217_Ab42_lo...,Intercept 2.151057e-16 pTau217_Ab4...,27,1.0,NaN,0.725136,2.151057e-16,NaN,...,NaN,-0.263960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,VOL_TotalGrayz,pTau217,Intercept 1.000000 pTau217_Ab42_lo...,Intercept 1.387779e-17 pTau217_Ab4...,27,1.0,NaN,0.906817,1.387779e-17,NaN,...,NaN,-0.282799,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,VOL_Ventricles_Lz,pTau217,Intercept 1.000000 pTau217_Ab42_lo...,Intercept 2.081668e-17 pTau217_Ab4...,27,1.0,NaN,0.621435,2.081668e-17,NaN,...,NaN,0.187538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,VOL_Ventricles_Rz,pTau217,Intercept 1.000000 pTau217_Ab42_lo...,Intercept 2.602085e-16 pTau217_Ab4...,27,1.0,NaN,0.297138,2.602085e-16,NaN,...,NaN,0.390285,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
comp_new_para.loc[(comp_new_para['ratioAb42_model_p_val_x_var'] < 0.05) & (comp_new_para['new_para_model_p_val_x_var'] < 0.05), ['x_var', 'y_var', 
'ratioAb42_model_p_val_x_var', 'new_para_model_p_val_x_var',
'ratioAb42_model_B_val_x_var', 'new_para_model_B_val_x_var', 
'ratioAb42_model_n_baseline','new_para_model_n_baseline']]

,x_var,y_var,ratioAb42_model_p_val_x_var,new_para_model_p_val_x_var,ratioAb42_model_B_val_x_var,new_para_model_B_val_x_var,ratioAb42_model_n_baseline,new_para_model_n_baseline
15,pTau181,pTau217,0.004133,0.003276,0.449087,0.489856,39,34
30,pTau217,bvrt,0.002598,0.005452,-0.490103,-0.481772,33,28
32,pTau217,tma,0.013732,0.049854,0.412498,0.381088,31,27
5,pTau217,Par_gm,0.001689,0.011858,-0.492376,-0.432901,38,33
7,pTau217,Temp_gm,0.000963,0.016024,-0.514007,-0.416077,38,33
24,pTau217,pTau181,0.004133,0.003276,0.449087,0.489856,39,34
25,pTau217,pTau231,0.031501,0.003210,0.359071,0.497659,36,33
16,pTau231,pTau217,0.031501,0.003210,0.359071,0.497659,36,33


## x_var significance

In [23]:
comp_new_para.loc[(comp_new_para['ratioAb42_model_p_val_x_var'] < 0.05) & (comp_new_para['new_para_model_p_val_x_var'] >= 0.05), ['x_var', 'y_var', 
'ratioAb42_model_p_val_x_var', 'new_para_model_p_val_x_var',
'ratioAb42_model_B_val_x_var', 'new_para_model_B_val_x_var', 
'ratioAb42_model_n_baseline','new_para_model_n_baseline']]

,x_var,y_var,ratioAb42_model_p_val_x_var,new_para_model_p_val_x_var,ratioAb42_model_B_val_x_var,new_para_model_B_val_x_var,ratioAb42_model_n_baseline,new_para_model_n_baseline
13,GFAP,pTau217,0.012522,0.160224,0.396230,0.246323,39,34
17,GFAP_aa,pTau217,0.010307,0.127015,0.416667,0.275468,37,32
31,pTau217,dstotal,0.044728,0.163501,-0.346448,-0.265756,34,29
4,pTau217,Precun_gm,0.044767,0.062075,-0.327454,-0.328377,38,33
1,pTau217,SensMot_gm,0.033251,0.086498,-0.346181,-0.303021,38,33
22,pTau217,GFAP,0.012522,0.160224,0.396230,0.246323,39,34
26,pTau217,GFAP_aa,0.010307,0.127015,0.416667,0.275468,37,32
115,pTau217,VOL_InfParSupra_Lz,0.004959,0.735875,-0.429074,-0.062011,32,27
116,pTau217,VOL_InfParSupra_Rz,0.035118,0.964642,-0.373721,-0.008955,32,27
56,pTau217,VOL_InfPar_Lz,0.018954,0.969799,-0.376511,0.007081,32,27


In [24]:
comp_new_para.loc[(comp_new_para['ratioAb42_model_p_val_x_var'] >= 0.05) & (comp_new_para['new_para_model_p_val_x_var'] < 0.05), 
['x_var', 'y_var',
'ratioAb42_model_p_val_x_var', 'new_para_model_p_val_x_var',
'ratioAb42_model_B_val_x_var', 'new_para_model_B_val_x_var', 
'ratioAb42_model_n_baseline','new_para_model_n_baseline']]

,x_var,y_var,ratioAb42_model_p_val_x_var,new_para_model_p_val_x_var,ratioAb42_model_B_val_x_var,new_para_model_B_val_x_var,ratioAb42_model_n_baseline,new_para_model_n_baseline
10,Ab40,pTau217,0.600249,0.000783,-0.089037,-0.548494,37,34
11,Ab42,pTau217,0.433116,0.000142,-0.138977,-0.606853,34,34
19,pTau217,Ab40,0.600249,0.000783,-0.089037,-0.548494,37,34
20,pTau217,Ab42,0.433116,0.000142,-0.138977,-0.606853,34,34
84,pTau217,VOL_AntCingulate_Lz,0.219084,0.043749,0.223382,0.390970,32,27
66,pTau217,VOL_Insula_Lz,0.427073,0.015863,0.145435,0.459634,32,27
67,pTau217,VOL_Insula_Rz,0.276686,0.019840,0.198268,0.445594,32,27
50,pTau217,VOL_MidTemp_Lz,0.570045,0.009827,-0.104281,0.516237,32,27


In [25]:
(comp_new_para['ratioAb42_model_p_val_x_var']  - comp_new_para['new_para_model_p_val_x_var']).describe().round(3)

count    116.000
mean      -0.097
std        0.353
min       -0.951
25%       -0.276
50%       -0.089
75%        0.112
max        0.817
dtype: float64

In [26]:
comp_new_para.loc[(abs(comp_new_para['ratioAb42_model_p_val_x_var']  - comp_new_para['new_para_model_p_val_x_var'])> 0.1), ['x_var', 'y_var', 'ratioAb42_model_p_val_x_var',
       'ratioAb42_model_B_val_x_var', 'ratioAb42_model_n_baseline', 'new_para_model_p_val_x_var', 'new_para_model_B_val_x_var',
       'new_para_model_n_baseline']].sort_values('new_para_model_p_val_x_var')

,x_var,y_var,ratioAb42_model_p_val_x_var,ratioAb42_model_B_val_x_var,ratioAb42_model_n_baseline,new_para_model_p_val_x_var,new_para_model_B_val_x_var,new_para_model_n_baseline
20,pTau217,Ab42,0.433116,-0.138977,34,0.000142,-0.606853,34
11,Ab42,pTau217,0.433116,-0.138977,34,0.000142,-0.606853,34
10,Ab40,pTau217,0.600249,-0.089037,37,0.000783,-0.548494,34
19,pTau217,Ab40,0.600249,-0.089037,37,0.000783,-0.548494,34
50,pTau217,VOL_MidTemp_Lz,0.570045,-0.104281,32,0.009827,0.516237,27
...,...,...,...,...,...,...,...,...
99,pTau217,VOL_Frontal_Lz,0.462486,-0.134652,32,0.958036,0.010629,27
46,pTau217,VOL_Fusi_Lz,0.191986,-0.236770,32,0.963078,-0.009351,27
116,pTau217,VOL_InfParSupra_Rz,0.035118,-0.373721,32,0.964642,-0.008955,27
56,pTau217,VOL_InfPar_Lz,0.018954,-0.376511,32,0.969799,0.007081,27


## covariate significance

In [27]:
# those where covariate significance differs between a and b
a_sig = comp_new_para[['ratioAb42_' + ele for ele in covariate_pval_columns]].notna().any(axis =1)
b_sig = comp_new_para[['new_para_' + ele for ele in covariate_pval_columns]].notna().any(axis =1)

temp = comp_new_para[a_sig != b_sig].drop(columns = comp_new_para.columns[comp_new_para.columns.str.contains('B_val|Intercept|baseline|type|p_val_x_var')]).dropna(axis = 1, how = 'all')
temp = temp[temp.columns.sort_values()]
temp = temp.loc[temp.isna().sort_values(temp.columns.to_list()).index, :]
temp

,new_para_model_p_val_Education_years,new_para_model_p_val_age,new_para_model_p_val_sex[T.M],ratioAb42_model_p_val_age,ratioAb42_model_p_val_apoe4_carrier[T.Yes],ratioAb42_model_p_val_sex[T.M],x_var,y_var
50,0.007396,0.017757,NaN,NaN,NaN,NaN,pTau217,VOL_MidTemp_Lz
51,0.046259,NaN,0.031,NaN,NaN,NaN,pTau217,VOL_MidTemp_Rz
70,NaN,0.010897,NaN,NaN,NaN,NaN,pTau217,VOL_MidFront_Lz
105,NaN,0.014246,NaN,NaN,NaN,NaN,pTau217,VOL_MidSupFrontal_Lz
54,NaN,0.021432,NaN,NaN,NaN,NaN,pTau217,VOL_Precun_Lz
32,NaN,NaN,NaN,0.034826,NaN,NaN,pTau217,tma
98,NaN,NaN,NaN,0.017752,NaN,NaN,pTau217,VOL_Parietal_Rz
61,NaN,NaN,NaN,NaN,0.022287,NaN,pTau217,VOL_SupraMarg_Rz
21,NaN,NaN,NaN,NaN,NaN,0.029434,pTau217,Ab42_40
88,NaN,NaN,NaN,NaN,NaN,0.015007,pTau217,VOL_Entorhinal_Lz


# Composite vs New Para

In [28]:
comp_new_para = pd.merge(results_composite, results_new_para, how = 'inner')
comp_new_para

NameError: name 'results_composite' is not defined

## x_var significance

In [ ]:
comp_new_para.loc[(comp_new_para['composite_model_p_val_x_var'] < 0.05) & (comp_new_para['new_para_model_p_val_x_var'] >= 0.05), ['x_var', 'y_var', 'composite_model_p_val_x_var',
       'composite_model_B_val_x_var', 'composite_model_n_baseline', 'new_para_model_p_val_x_var', 'new_para_model_B_val_x_var',
       'new_para_model_n_baseline']]

In [ ]:
comp_new_para.loc[(comp_new_para['composite_model_p_val_x_var'] >= 0.05) & (comp_new_para['new_para_model_p_val_x_var'] < 0.05), ['x_var', 'y_var', 'composite_model_p_val_x_var',
       'composite_model_B_val_x_var', 'composite_model_n_baseline', 'new_para_model_p_val_x_var', 'new_para_model_B_val_x_var',
       'new_para_model_n_baseline']]

In [ ]:
(comp_new_para['composite_model_p_val_x_var']  - comp_new_para['new_para_model_p_val_x_var']).describe().round(3)

In [ ]:
comp_new_para.loc[(abs(comp_new_para['composite_model_p_val_x_var']  - comp_new_para['new_para_model_p_val_x_var'])> 0.1), ['x_var', 'y_var', 'composite_model_p_val_x_var',
       'composite_model_B_val_x_var', 'composite_model_n_baseline', 'new_para_model_p_val_x_var', 'new_para_model_B_val_x_var',
       'new_para_model_n_baseline']].sort_values('new_para_model_p_val_x_var')

## covariate significance

In [ ]:
# those where covariate significance differs between a and b
a_sig = comp_new_para[['composite_' + ele for ele in covariate_pval_columns]].notna().any(axis =1)
b_sig = comp_new_para[['new_para_' + ele for ele in covariate_pval_columns]].notna().any(axis =1)

temp = comp_new_para[a_sig != b_sig].drop(columns = comp_new_para.columns[comp_new_para.columns.str.contains('B_val|Intercept|baseline|type|p_val_x_var')]).dropna(axis = 1, how = 'all')
temp = temp[temp.columns.sort_values()]
temp = temp.loc[temp.isna().sort_values(temp.columns.to_list()).index, :]
temp

# new_para vs subset new_para

In [ ]:
new_para_new_para = pd.merge(results_new_para, results_subset_new_para, how = 'inner')
new_para_new_para

## x_var significance

In [ ]:
new_para_new_para[(new_para_new_para['new_para_model_p_val_x_var'] < 0.05) & (new_para_new_para['subset_new_para_model_p_val_x_var'] >= 0.05)]

In [ ]:
new_para_new_para[(new_para_new_para['new_para_model_p_val_x_var'] >= 0.05) & (new_para_new_para['subset_new_para_model_p_val_x_var'] < 0.05)]

In [ ]:
(new_para_new_para['new_para_model_p_val_x_var']  - new_para_new_para['subset_new_para_model_p_val_x_var']).describe().round(3)

## covariate significance

In [ ]:
# those where covariate significance differs between a and b
a_sig = new_para_new_para[['new_para_' + ele for ele in covariate_pval_columns]].notna().any(axis =1)
b_sig = new_para_new_para[['subset_new_para_' + ele for ele in covariate_pval_columns]].notna().any(axis =1)

temp = new_para_new_para[a_sig != b_sig].drop(columns = new_para_new_para.columns[new_para_new_para.columns.str.contains('B_val|Intercept|baseline|type|p_val_x_var')]).dropna(axis = 1, how = 'all')
temp = temp[temp.columns.sort_values()]
temp.loc[temp.isna().sort_values(temp.columns.to_list()).index, :]
temp

# Ras vs new para

In [ ]:
ras_new_para = pd.merge(results_ras, results_new_para, how = 'inner')
ras_new_para

## x_var significance

In [ ]:
ras_new_para[(ras_new_para['ras_model_p_val_x_var'] < 0.05) & (ras_new_para['new_para_model_p_val_x_var'] >= 0.05)]

In [ ]:
ras_new_para[(ras_new_para['ras_model_p_val_x_var'] >= 0.05) & (ras_new_para['new_para_model_p_val_x_var'] < 0.05)]

In [ ]:
(ras_new_para['ras_model_p_val_x_var']  - ras_new_para['new_para_model_p_val_x_var']).describe().round(3)

## covariate significance

In [ ]:
# those where covariate significance differs between a and b
a_sig = ras_new_para[['new_para_' + ele for ele in covariate_pval_columns]].notna().any(axis =1)
b_sig = ras_new_para[['ras_' + ele for ele in covariate_pval_columns]].notna().any(axis =1)

temp = ras_new_para[a_sig != b_sig].drop(columns = ras_new_para.columns[ras_new_para.columns.str.contains('B_val|Intercept|baseline|type|p_val_x_var')]).dropna(axis = 1, how = 'all')
temp = temp[temp.columns.sort_values()]
temp.loc[temp.isna().sort_values(temp.columns.to_list()).index, :]
temp

# Composite vs Old Para

In [ ]:
comp_para = pd.merge(results_composite, results_para, how = 'inner')
comp_para

## x_var significance

In [ ]:
comp_para[(comp_para['composite_model_p_val_x_var'] < 0.05) & (comp_para['para_model_p_val_x_var'] >= 0.05)]

In [ ]:
comp_para[(comp_para['composite_model_p_val_x_var'] >= 0.05) & (comp_para['para_model_p_val_x_var'] < 0.05)]

In [ ]:
(comp_para['composite_model_p_val_x_var']  - comp_para['para_model_p_val_x_var']).describe().round(3)

## covariate significance

In [ ]:
# those where covariate significance differs between a and b
a_sig = comp_para[['composite_' + ele for ele in covariate_pval_columns]].notna().any(axis =1)
b_sig = comp_para[['para_' + ele for ele in covariate_pval_columns]].notna().any(axis =1)

temp = comp_para[a_sig != b_sig].drop(columns = comp_para.columns[comp_para.columns.str.contains('B_val|Intercept|baseline|type|p_val_x_var')]).dropna(axis = 1, how = 'all')
temp = temp[temp.columns.sort_values()]
temp.loc[temp.isna().sort_values(temp.columns.to_list()).index, :]
temp

# New Para with and without RIL8

In [ ]:
comp_new_para = pd.merge(results_orig_new_para, results_new_para, how = 'inner')
comp_new_para

## x_var significance

In [ ]:
comp_new_para.loc[(comp_new_para['orig_new_para_model_p_val_x_var'] < 0.05) & (comp_new_para['new_para_model_p_val_x_var'] >= 0.05), ['x_var', 'y_var', 'orig_new_para_model_p_val_x_var',
       'orig_new_para_model_B_val_x_var', 'orig_new_para_model_n_baseline', 'new_para_model_p_val_x_var', 'new_para_model_B_val_x_var',
       'new_para_model_n_baseline']]

In [ ]:
comp_new_para.loc[(comp_new_para['orig_new_para_model_p_val_x_var'] >= 0.05) & (comp_new_para['new_para_model_p_val_x_var'] < 0.05), ['x_var', 'y_var', 'orig_new_para_model_p_val_x_var',
       'orig_new_para_model_B_val_x_var', 'orig_new_para_model_n_baseline', 'new_para_model_p_val_x_var', 'new_para_model_B_val_x_var',
       'new_para_model_n_baseline']]

In [ ]:
(comp_new_para['orig_new_para_model_p_val_x_var']  - comp_new_para['new_para_model_p_val_x_var']).describe().round(3)

In [ ]:
comp_new_para.loc[(abs(comp_new_para['orig_new_para_model_p_val_x_var']  - comp_new_para['new_para_model_p_val_x_var'])> 0.1), ['x_var', 'y_var', 'orig_new_para_model_p_val_x_var',
       'orig_new_para_model_B_val_x_var', 'orig_new_para_model_n_baseline', 'new_para_model_p_val_x_var', 'new_para_model_B_val_x_var',
       'new_para_model_n_baseline']].sort_values('new_para_model_p_val_x_var')

## covariate significance

In [ ]:
# those where covariate significance differs between a and b
a_sig = comp_new_para[['orig_new_para_' + ele for ele in covariate_pval_columns]].notna().any(axis =1)
b_sig = comp_new_para[['new_para_' + ele for ele in covariate_pval_columns]].notna().any(axis =1)

temp = comp_new_para[a_sig != b_sig].drop(columns = comp_new_para.columns[comp_new_para.columns.str.contains('B_val|Intercept|baseline|type|p_val_x_var')]).dropna(axis = 1, how = 'all')
temp = temp[temp.columns.sort_values()]
temp.loc[temp.isna().sort_values(temp.columns.to_list()).index, :]
temp